# 说明：

本节用tf里的keras，来搭建最简单的网络处理“**序列问题**”；使用“**电影评论数据集（内置）**”；用最简单的“**全连接网络**”
- tf.keras.models.xxx：模型相关
- tf.keras.layers.xxx：各种层的函数
- tf.keras.preprocessing.xxx：各种数据预处理操作
- tf.keras.optimizers.xxx：各种实例化优化器 —— 可以自定义里面的超参数！

注意点：
- dropout是训练时有，但测试时所有神经元都上场的！
- 展平层中：keras.layers.GlobalAveragePooling1D()比keras.layers.Flatten()更好！节省要训练的参数！
- Dense层中的各种regularizer正则化参数可以试一试，优化网络！

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [28]:
# 数据导入：
data = keras.datasets.imdb
(x_train, y_train), (x_test, y_test) = data.load_data()

In [29]:
x_train.shape, x_test.shape

((25000,), (25000,))

In [30]:
# 训练集中有25000段话，即里面是以“话”为单位的；
# 每段话中的有很多单词，但每个单词转编成“已排好序列”中的索引值！
type(x_train[122]), len(x_train[122])  # 第一句话中有218个单词！

(list, 60)

In [31]:
x_train[122][:10]  # 看前10个

[1, 4, 7181, 7, 14, 22, 26, 4, 5149, 997]

In [32]:
# 用下面这句话，可以获得数字编码对应的单词：下载太慢
# data.get_word_index()

In [33]:
# 上面是直接默认“以排好序列”直接进行，那么就有25000个单词，这太多了！可以人为设定
max_word = 250  # 一句话中，索引值大于1000的单词全部用“相同的某一个”索引值代替！
(x_train, y_train), (x_test, y_test) = data.load_data(num_words=max_word)

In [34]:
type(x_train[122]), len(x_train[122])

(list, 60)

In [35]:
x_train[122][:10]  # 很明显，大于250的索引值，都用索引值2代替了！

[1, 4, 2, 7, 14, 22, 26, 4, 2, 2]

### 预处理：

要做的预处理有：
- 句子长度统一
- 里面的数值要做处理，不能直接用：tf.keras.layers.Embedding() —— 拓展出一个更高维！

In [39]:
# 所有的句子长度不一，现统一到每句话都是300个词：
x_train = keras.preprocessing.sequence.pad_sequences(x_train, 300)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, 300)

### 网络搭建：

In [61]:
model = keras.models.Sequential()

In [62]:
model.add( keras.layers.Embedding(250, 50, input_length = 300) )  # 索引范围250，拓展出的维度总长50，每句话300长度
model.add( keras.layers.Flatten() )
# model.output_shape
# keras.layers.GlobalAveragePooling1D()替代Flatten()层更好！减少待训练参数！
model.add( keras.layers.Dense(16, activation = 'relu') )
model.add( keras.layers.Dense(1, activation = 'sigmoid') )  # 输出层：2分类

In [57]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 300, 50)           12500     
_________________________________________________________________
flatten_3 (Flatten)          (None, 15000)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 16)                240016    
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 17        
Total params: 252,533
Trainable params: 252,533
Non-trainable params: 0
_________________________________________________________________


### 网络编译：

In [58]:
model.compile(
    optimizer = keras.optimizers.Adam( lr = 0.001 ),
    loss = 'binary_crossentropy',
    metrics = ['acc']
)

In [59]:
model.fit( 
    x_train, 
    y_train, 
    epochs = 5,  
    batch_size = 256,  # 每次送入256张
    validation_data = (x_test, y_test)
)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 4s 177us/sample - loss: 0.6351 - acc: 0.6249 - val_loss: 0.5154 - val_acc: 0.7469
Epoch 2/5
25000/25000 [==============================] - 3s 129us/sample - loss: 0.4591 - acc: 0.7848 - val_loss: 0.4728 - val_acc: 0.7743
Epoch 3/5
25000/25000 [==============================] - 3s 127us/sample - loss: 0.4022 - acc: 0.8171 - val_loss: 0.4812 - val_acc: 0.7691
Epoch 4/5
25000/25000 [==============================] - 3s 123us/sample - loss: 0.3483 - acc: 0.8500 - val_loss: 0.4956 - val_acc: 0.7664
Epoch 5/5
25000/25000 [==============================] - 3s 124us/sample - loss: 0.2956 - acc: 0.8800 - val_loss: 0.5339 - val_acc: 0.7575


In [60]:
model.evaluate(x_test, y_test)

25000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.5339197243642807, 0.75748]